<a href="https://colab.research.google.com/github/arny617/MLNanodegree/blob/master/Crypto_SentimentAnalysis_RandomForest_UsingTrigrams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.corpus import stopwords as sw
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import numpy as np
import nltk
import os
nltk.download('wordnet')
import pandas as pd
import numpy as np
import pickle
import joblib

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [3]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [4]:
!pip install pysentiment

     |████████████████████████████████| 2.8MB 6.1MB/s 
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.


In [5]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
import gensim

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [6]:
!pip install OpenBlender

     |████████████████████████████████| 61kB 3.0MB/s 
     |████████████████████████████████| 245kB 6.2MB/s 
  Created wheel for OpenBlender: filename=OpenBlender-2.6-cp36-none-any.whl size=6608 sha256=ba7e43f0e734ca75d36b3f0c41df8d8131d7ea94ceb60360d8583d271b38e7e2
  Stored in directory: /root/.cache/pip/wheels/fd/29/7b/377afef16a792ed174a9ade5a6707ef87843e214476d0c8205
Successfully built OpenBlender


In [7]:
import OpenBlender

In [8]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [9]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
ROOT = "/content/gdrive/My Drive/Colab Notebooks/NLP/Fast.ai_IntroToNLP/"
os.chdir(ROOT)

Mounted at /content/gdrive


### Get Data using OpenBlender

In [ ]:
api_token = '5fc5ad47951629617230bee5MDHzrVvlWhz1MZm92QeQqc5DrybPBT'

In [ ]:
action = 'API_getObservationsFromDataset'# ANCHOR: 'Bitcoin vs USD'
  
parameters = { 
    'token' : api_token,
    'id_dataset' : '5d4c3af79516290b01c83f51',
    'date_filter':{"start_date" : "2020-01-01",
                   "end_date" : "2020-08-29"} }
df = pd.read_json(json.dumps(OpenBlender.call(action, parameters)['sample']), convert_dates=False, convert_axes=False).sort_values('timestamp', ascending=False)
df.reset_index(drop=True, inplace=True)
df['date'] = [OpenBlender.unixToDate(ts, timezone = 'GMT') for ts in df.timestamp]
df = df.drop('timestamp', axis = 1)

Task ID: '5fc84fb90895fafb4a9d8d4b'.
Total estimated consumption: 5832.61 processing units.
Task confirmed. Starting download..
100.0 % completed.


In [ ]:
df['log_ret'] = np.log(df['price']) - np.log(df['open'])

In [ ]:
df['target'] = [1 if log_diff > 0 else 0 for log_diff in df['log_ret']]

In [ ]:
format = '%d-%m-%Y %H:%M:%S'
timezone = 'GMT'
df['u_timestamp'] = OpenBlender.dateToUnix(df['date'], 
                                           date_format = format, 
                                           timezone = timezone)
df = df[['date', 'u_timestamp', 'price', 'target']]
df.head()

,date,u_timestamp,price,target
0,28-08-2020 17:00:00,1.598634e+09,11545.0,1
1,27-08-2020 17:00:00,1.598548e+09,11343.0,0
2,26-08-2020 17:00:00,1.598461e+09,11471.0,1
3,25-08-2020 17:00:00,1.598375e+09,11341.0,0
4,23-08-2020 17:00:00,1.598202e+09,11663.0,0


In [ ]:
blend_source = {
                'id_dataset':'5ea2039095162936337156c9',
                'feature' : 'text'
            }
# Now, let's 'timeBlend' it to our dataset
df_blend = OpenBlender.timeBlend( token = api_token,
                                  anchor_ts = df.u_timestamp,
                                  blend_source = blend_source,
                                  blend_type = 'agg_in_intervals',
                                  interval_size = 60 * 60 * 24,
                                  direction = 'time_prior',
                                  interval_output = 'list',
                                  missing_values = 'raw')
df = pd.concat([df, df_blend.loc[:, df_blend.columns != 'u_timestamp']], axis = 1)
df.head()

Task ID: '5fc850470895fafb4a9d8d4c'.
Total estimated consumption: 9440.0 processing units.
Task confirmed. Starting download..
100%


,date,u_timestamp,price,target,timestamp,BITCOIN_NE.text_COUNT_last1days,BITCOIN_NE.text_last1days
0,28-08-2020 17:00:00,1.598634e+09,11545.0,1,1577898000,3,[etoro ceo yoni assia on reaching 12 million u...
1,27-08-2020 17:00:00,1.598548e+09,11343.0,0,1577984400,7,[government confirms crypto profits not taxabl...
2,26-08-2020 17:00:00,1.598461e+09,11471.0,1,1578070800,8,[ticker tool uses the bch blockchain to provid...
3,25-08-2020 17:00:00,1.598375e+09,11341.0,0,1578157200,6,[crypto fundraising is changing again in 2020 ...
4,23-08-2020 17:00:00,1.598202e+09,11663.0,0,1578243600,5,[subhash chandra garg on the future of crypto ...


In [ ]:
df['text'] = df['BITCOIN_NE.text_last1days'].apply(lambda x:"xxeos ".join(x))

In [ ]:
vader_analyzer = SentimentIntensityAnalyzer()

In [ ]:
df["text_polarity"] = df["text"].apply(lambda x : vader_analyzer.polarity_scores(x))
df["text_subjectivity"] = df["text"].apply(lambda x : TextBlob(x).sentiment[1])

In [ ]:
# BTC Text Vectorizer
blend_source = { 
                'id_textVectorizer':'5f739fe7951629649472e167'
               }
df_blend = OpenBlender.timeBlend( token = api_token,
                                  anchor_ts = df.u_timestamp,
                                  blend_source = blend_source,
                                  blend_type = 'agg_in_intervals',
                                  interval_size = 60 * 60 * 24,
                                  direction = 'time_prior',
                                  interval_output = 'list',
                                  missing_values = 'raw').add_prefix('VEC.')

Task ID: '5fc857840895fafb4a9d8d4c'.
Total estimated consumption: 10479.43 processing units.
Task confirmed. Starting download..
100%


In [ ]:
df_anchor = pd.concat([df, df_blend.loc[:, df_blend.columns != 'timestamp']], axis = 1)
df_anchor.head()

,date,u_timestamp,price,target,timestamp,BITCOIN_NE.text_COUNT_last1days,BITCOIN_NE.text_last1days,text,text_polarity,text_subjectivity,VEC.block reward,VEC.shop,VEC.code,VEC.virgin,VEC.chain,VEC.miner,VEC.demand,VEC.dollar,VEC.month,VEC.funding debate,VEC.kicks,VEC.httpbitcoincom wallet,VEC.battle,VEC.chair,VEC.issues,VEC.chinese,VEC.electricity,VEC.billion loans,VEC.defi flash,VEC.group,VEC.countdown,VEC.list,VEC.charge,VEC.program,VEC.stimulus checks,VEC.helps,VEC.encrypted,VEC.breach,VEC.lawmakers,VEC.worth,...,VEC.coinjoin,VEC.star,VEC.coindeal,VEC.congress,VEC.bitcoin mining,VEC.grayscale,VEC.printing,VEC.return,VEC.regulations,VEC.crypto traders,VEC.global economy,VEC.allowing,VEC.ceos,VEC.authorities,VEC.funding,VEC.plustoken,VEC.stocks,VEC.presidential,VEC.pool,VEC.billion,VEC.sanctions,VEC.bch,VEC.assets,VEC.calls,VEC.tank,VEC.space,VEC.youtube,VEC.daily,VEC.reports,VEC.2000,VEC.safe,VEC.heats,VEC.profits,VEC.volumes,VEC.time,VEC.laws,VEC.expands,VEC.integrates,VEC.digital gold,VEC.minister
0,28-08-2020 17:00:00,1.598634e+09,11545.0,1,1577898000,3,[etoro ceo yoni assia on reaching 12 million u...,etoro ceo yoni assia on reaching 12 million us...,"{'neg': 0.0, 'neu': 0.954, 'pos': 0.046, 'comp...",0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,27-08-2020 17:00:00,1.598548e+09,11343.0,0,1577984400,7,[government confirms crypto profits not taxabl...,government confirms crypto profits not taxable...,"{'neg': 0.032, 'neu': 0.787, 'pos': 0.181, 'co...",0.462500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,26-08-2020 17:00:00,1.598461e+09,11471.0,1,1578070800,8,[ticker tool uses the bch blockchain to provid...,ticker tool uses the bch blockchain to provide...,"{'neg': 0.122, 'neu': 0.785, 'pos': 0.093, 'co...",0.254167,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,25-08-2020 17:00:00,1.598375e+09,11341.0,0,1578157200,6,[crypto fundraising is changing again in 2020 ...,crypto fundraising is changing again in 2020 x...,"{'neg': 0.063, 'neu': 0.847, 'pos': 0.09, 'com...",0.518182,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,23-08-2020 17:00:00,1.598202e+09,11663.0,0,1578243600,5,[subhash chandra garg on the future of crypto ...,subhash chandra garg on the future of crypto x...,"{'neg': 0.049, 'neu': 0.772, 'pos': 0.179, 'co...",0.319886,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_anchor.to_csv("BTC_Final.csv")

### Load Pre Saved Data

In [42]:
df_anchor = pd.read_csv("BTC_Final.csv",usecols=[1,2,3,4,5,6,7,8,9,10])

In [43]:
df_anchor.head()

,date,u_timestamp,price,target,timestamp,BITCOIN_NE.text_COUNT_last1days,BITCOIN_NE.text_last1days,text,text_polarity,text_subjectivity
0,28-08-2020 17:00:00,1.598634e+09,11545.0,1,1577898000,3,['etoro ceo yoni assia on reaching 12 million ...,etoro ceo yoni assia on reaching 12 million us...,"{'neg': 0.0, 'neu': 0.954, 'pos': 0.046, 'comp...",0.000000
1,27-08-2020 17:00:00,1.598548e+09,11343.0,0,1577984400,7,['government confirms crypto profits not taxab...,government confirms crypto profits not taxable...,"{'neg': 0.032, 'neu': 0.787, 'pos': 0.181, 'co...",0.462500
2,26-08-2020 17:00:00,1.598461e+09,11471.0,1,1578070800,8,['ticker tool uses the bch blockchain to provi...,ticker tool uses the bch blockchain to provide...,"{'neg': 0.122, 'neu': 0.785, 'pos': 0.093, 'co...",0.254167
3,25-08-2020 17:00:00,1.598375e+09,11341.0,0,1578157200,6,['crypto fundraising is changing again in 2020...,crypto fundraising is changing again in 2020 x...,"{'neg': 0.063, 'neu': 0.847, 'pos': 0.09, 'com...",0.518182
4,23-08-2020 17:00:00,1.598202e+09,11663.0,0,1578243600,5,['subhash chandra garg on the future of crypto...,subhash chandra garg on the future of crypto x...,"{'neg': 0.049, 'neu': 0.772, 'pos': 0.179, 'co...",0.319886


In [28]:
#df_anchor["text_polarity"] = df_anchor["text_polarity"].astype('object')

In [38]:
from ast import literal_eval

In [39]:
def try_literal_eval(s):
    try:
        return literal_eval(s)
    except ValueError:
        return s

In [44]:
df_anchor["text_polarity"] = df_anchor["text_polarity"].apply(lambda x:try_literal_eval(x))

In [41]:
df_anchor["text_polarity_new"]

0      {'neg': 0.0, 'neu': 0.954, 'pos': 0.046, 'comp...
1      {'neg': 0.032, 'neu': 0.787, 'pos': 0.181, 'co...
2      {'neg': 0.122, 'neu': 0.785, 'pos': 0.093, 'co...
3      {'neg': 0.063, 'neu': 0.847, 'pos': 0.09, 'com...
4      {'neg': 0.049, 'neu': 0.772, 'pos': 0.179, 'co...
                             ...                        
231    {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...
232    {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...
233    {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...
234    {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...
235    {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...
Name: text_polarity_new, Length: 236, dtype: object

In [45]:
df_anchor["text_posneg"] = df_anchor["text_polarity"].apply(lambda x : 0 if x["neg"]==0.0 else x["pos"]/x["neg"]) 


In [46]:
df_anchor["text_polarity"] = df_anchor["text_polarity"].apply(lambda x : x["compound"]) 

In [52]:
import nltk
from nltk.corpus import stopwords as sw
from nltk.stem import WordNetLemmatizer 
from nltk.stem import SnowballStemmer
import gensim
from gensim.utils import simple_preprocess
from tqdm import tqdm, tqdm_notebook
from tqdm import notebook

In [53]:
def clean_text(text_blob):
    """
    This function aims to perform a series of text processing tasks - 
    For each paragraph
        a)basic pre-processing and removal of punctuations
        b)perform POS tagiing
        c)remove stopwords, non alphabetical character sequences and proper nouns
    
    Input : Simple raw text corpus
    Output : list of processed text
    """
    stopwords = sw.words("english")
    ps = SnowballStemmer("english")
    #ps = PorterStemmer()
    para = ' '.join(simple_preprocess(text_blob,deacc = True)) #Basic preprocessing and removal of punctuations
    tagged_list = nltk.tag.pos_tag(para.split())
    fin_text_blob = []
    for word,tag in tagged_list:
        if((word.lower() not in stopwords) and (word.isalpha()) and (len(word) >= 3) and (tag != "NNP" and tag != "NNPS")):
            fin_text_blob.extend([ps.stem(word.lower())])
    return(fin_text_blob) #This function hence returns the original text blob, processed 

In [55]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [57]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [60]:
text_list = []
for i in notebook.tqdm(range(df_anchor.shape[0])):
    text_blob = clean_text(str(df_anchor.text[i]))
    text_list.extend([text_blob])

In [66]:
#flat_list = [item for sublist in text_list for item in sublist]

In [68]:
import gensim
bigram = gensim.models.Phrases(text_list, min_count=5, threshold=50) #Generating bigrams
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [69]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [70]:
data_words_bigrams = make_bigrams(text_list) 

In [72]:
trigram = gensim.models.Phrases(data_words_bigrams, min_count=5, threshold=50)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [74]:
def make_trigrams(texts):
    return [trigram_mod[doc] for doc in texts]

In [75]:
data_words_trigrams = make_trigrams(data_words_bigrams) 

In [88]:
df_anchor['text_new'] = [" ".join(item) for item in data_words_trigrams]

In [90]:
count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = count_vect.fit_transform(df_anchor['text_new'].values.astype('U'))

In [91]:
doc_term_matrix

<236x1312 sparse matrix of type '<class 'numpy.int64'>'
	with 8141 stored elements in Compressed Sparse Row format>

In [98]:
features = count_vect.get_feature_names()
features = list(map('VEC_'.__add__,features))

In [99]:
df_res = pd.DataFrame(doc_term_matrix.toarray(), columns=features)

In [100]:
df_res.head()

,VEC_abc,VEC_acceler,VEC_accept,VEC_access,VEC_accord,VEC_account,VEC_accus,VEC_acquir,VEC_acquisit,VEC_act,VEC_action,VEC_activ,VEC_ad,VEC_adam,VEC_add,VEC_address,VEC_adopt,VEC_advertis,VEC_affect,VEC_africa,VEC_african,VEC_agenc,VEC_agent,VEC_aggreg,VEC_ahead,VEC_aid,VEC_aim,VEC_air,VEC_airdrop,VEC_airport,VEC_algorithm,VEC_alleg,VEC_allow,VEC_alltim,VEC_altcoin,VEC_altern,VEC_america,VEC_american,VEC_amid,VEC_aml,...,VEC_watchdog,VEC_wave,VEC_way,VEC_wealth,VEC_web,VEC_websit,VEC_week,VEC_wendi,VEC_west,VEC_whale,VEC_white,VEC_wild,VEC_win,VEC_wirecard,VEC_wirex,VEC_withdraw,VEC_withdrawn,VEC_woman,VEC_wordpress,VEC_world,VEC_worldwid,VEC_worri,VEC_worst,VEC_worth,VEC_wrap,VEC_xrp,VEC_yam,VEC_year,VEC_yearold,VEC_yield,VEC_york,VEC_young,VEC_youtub,VEC_yuan,VEC_zcash,VEC_zebpay,VEC_zec,VEC_zero,VEC_zhuoer,VEC_zone
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [101]:
df_new = df_anchor.merge(df_res,left_index=True,right_index=True)

In [102]:
df_new.head()

,date,u_timestamp,price,target,timestamp,BITCOIN_NE.text_COUNT_last1days,BITCOIN_NE.text_last1days,text,text_polarity,text_subjectivity,text_posneg,text_new,VEC_abc,VEC_acceler,VEC_accept,VEC_access,VEC_accord,VEC_account,VEC_accus,VEC_acquir,VEC_acquisit,VEC_act,VEC_action,VEC_activ,VEC_ad,VEC_adam,VEC_add,VEC_address,VEC_adopt,VEC_advertis,VEC_affect,VEC_africa,VEC_african,VEC_agenc,VEC_agent,VEC_aggreg,VEC_ahead,VEC_aid,VEC_aim,VEC_air,...,VEC_watchdog,VEC_wave,VEC_way,VEC_wealth,VEC_web,VEC_websit,VEC_week,VEC_wendi,VEC_west,VEC_whale,VEC_white,VEC_wild,VEC_win,VEC_wirecard,VEC_wirex,VEC_withdraw,VEC_withdrawn,VEC_woman,VEC_wordpress,VEC_world,VEC_worldwid,VEC_worri,VEC_worst,VEC_worth,VEC_wrap,VEC_xrp,VEC_yam,VEC_year,VEC_yearold,VEC_yield,VEC_york,VEC_young,VEC_youtub,VEC_yuan,VEC_zcash,VEC_zebpay,VEC_zec,VEC_zero,VEC_zhuoer,VEC_zone
0,28-08-2020 17:00:00,1.598634e+09,11545.0,1,1577898000,3,['etoro ceo yoni assia on reaching 12 million ...,etoro ceo yoni assia on reaching 12 million us...,0.2023,0.000000,0.000000,etoro ceo yoni assia reach million user crypto...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,27-08-2020 17:00:00,1.598548e+09,11343.0,0,1577984400,7,['government confirms crypto profits not taxab...,government confirms crypto profits not taxable...,0.9136,0.462500,5.656250,govern confirm crypto profit taxabl south_kore...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,26-08-2020 17:00:00,1.598461e+09,11471.0,1,1578070800,8,['ticker tool uses the bch blockchain to provi...,ticker tool uses the bch blockchain to provide...,-0.5647,0.254167,0.762295,ticker tool use bch blockchain provid reliabl ...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
3,25-08-2020 17:00:00,1.598375e+09,11341.0,0,1578157200,6,['crypto fundraising is changing again in 2020...,crypto fundraising is changing again in 2020 x...,0.1531,0.518182,1.428571,crypto fundrais chang proof key day return bit...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,23-08-2020 17:00:00,1.598202e+09,11663.0,0,1578243600,5,['subhash chandra garg on the future of crypto...,subhash chandra garg on the future of crypto x...,0.8126,0.319886,3.653061,subhash chandra garg futur crypto ross ulbrich...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [103]:
df_final = df_new[df_new.columns[~df_new.columns.isin(['date','timestamp','price','BITCOIN_NE.text_COUNT_last1days','BITCOIN_NE.text_last1days','text','VEC.source','text_new'])]]

In [104]:
df_final.head()

,u_timestamp,target,text_polarity,text_subjectivity,text_posneg,VEC_abc,VEC_acceler,VEC_accept,VEC_access,VEC_accord,VEC_account,VEC_accus,VEC_acquir,VEC_acquisit,VEC_act,VEC_action,VEC_activ,VEC_ad,VEC_adam,VEC_add,VEC_address,VEC_adopt,VEC_advertis,VEC_affect,VEC_africa,VEC_african,VEC_agenc,VEC_agent,VEC_aggreg,VEC_ahead,VEC_aid,VEC_aim,VEC_air,VEC_airdrop,VEC_airport,VEC_algorithm,VEC_alleg,VEC_allow,VEC_alltim,VEC_altcoin,...,VEC_watchdog,VEC_wave,VEC_way,VEC_wealth,VEC_web,VEC_websit,VEC_week,VEC_wendi,VEC_west,VEC_whale,VEC_white,VEC_wild,VEC_win,VEC_wirecard,VEC_wirex,VEC_withdraw,VEC_withdrawn,VEC_woman,VEC_wordpress,VEC_world,VEC_worldwid,VEC_worri,VEC_worst,VEC_worth,VEC_wrap,VEC_xrp,VEC_yam,VEC_year,VEC_yearold,VEC_yield,VEC_york,VEC_young,VEC_youtub,VEC_yuan,VEC_zcash,VEC_zebpay,VEC_zec,VEC_zero,VEC_zhuoer,VEC_zone
0,1.598634e+09,1,0.2023,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.598548e+09,0,0.9136,0.462500,5.656250,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.598461e+09,1,-0.5647,0.254167,0.762295,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
3,1.598375e+09,0,0.1531,0.518182,1.428571,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.598202e+09,0,0.8126,0.319886,3.653061,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [105]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score# We drop correlated features because with so many binary 
# ngram variables there's a lot of noise

corr_matrix = df_final.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

In [106]:
corr_matrix

,u_timestamp,target,text_polarity,text_subjectivity,text_posneg,VEC_abc,VEC_acceler,VEC_accept,VEC_access,VEC_accord,VEC_account,VEC_accus,VEC_acquir,VEC_acquisit,VEC_act,VEC_action,VEC_activ,VEC_ad,VEC_adam,VEC_add,VEC_address,VEC_adopt,VEC_advertis,VEC_affect,VEC_africa,VEC_african,VEC_agenc,VEC_agent,VEC_aggreg,VEC_ahead,VEC_aid,VEC_aim,VEC_air,VEC_airdrop,VEC_airport,VEC_algorithm,VEC_alleg,VEC_allow,VEC_alltim,VEC_altcoin,...,VEC_watchdog,VEC_wave,VEC_way,VEC_wealth,VEC_web,VEC_websit,VEC_week,VEC_wendi,VEC_west,VEC_whale,VEC_white,VEC_wild,VEC_win,VEC_wirecard,VEC_wirex,VEC_withdraw,VEC_withdrawn,VEC_woman,VEC_wordpress,VEC_world,VEC_worldwid,VEC_worri,VEC_worst,VEC_worth,VEC_wrap,VEC_xrp,VEC_yam,VEC_year,VEC_yearold,VEC_yield,VEC_york,VEC_young,VEC_youtub,VEC_yuan,VEC_zcash,VEC_zebpay,VEC_zec,VEC_zero,VEC_zhuoer,VEC_zone
u_timestamp,1.000000,0.078198,0.129950,0.236463,0.167387,0.115901,0.066068,0.039389,0.143596,0.008230,0.000393,0.003665,0.019009,0.091059,0.020138,0.079987,0.043276,0.022020,0.015346,0.093893,0.163510,0.002701,0.084980,0.043839,0.144465,0.174544,0.126182,0.009581,0.048081,0.066975,0.019713,0.006368,0.004494,0.010375,0.110647,0.003992,0.021853,0.037789,0.005143,0.021481,...,0.112064,0.013368,0.056107,0.021431,0.032546,0.024921,0.023213,0.121712,0.002827,0.056622,0.110647,0.002640,0.096442,0.111939,0.125757,0.109572,0.060665,0.125757,0.037274,0.025097,0.051022,0.002886,0.018668,0.113708,0.053485,0.054067,0.141967,0.072999,0.085230,0.163742,0.119002,0.026217,0.048848,0.146294,0.010682,0.031626,0.053910,0.110636,0.122130,0.085235
target,0.078198,1.000000,0.003190,0.007029,0.052693,0.010044,0.085648,0.034943,0.055801,0.007072,0.098361,0.016091,0.055801,0.085648,0.085648,0.099785,0.081797,0.075889,0.099792,0.114605,0.068949,0.055801,0.007072,0.040764,0.029090,0.040764,0.085648,0.085648,0.085648,0.010044,0.007072,0.033772,0.075889,0.010044,0.099792,0.046614,0.111604,0.074666,0.040764,0.136298,...,0.010044,0.099792,0.099785,0.029254,0.099792,0.099785,0.033726,0.029254,0.007072,0.038423,0.007072,0.105122,0.029090,0.055801,0.099792,0.032637,0.085648,0.085648,0.085648,0.075870,0.055801,0.029254,0.029254,0.084338,0.085648,0.061775,0.007072,0.066050,0.007072,0.010044,0.007072,0.007072,0.126570,0.011676,0.007072,0.029254,0.007072,0.111874,0.085648,0.029254
text_polarity,0.129950,0.003190,1.000000,0.001274,0.583386,0.054351,0.058027,0.072051,0.006972,0.037018,0.085059,0.005516,0.010643,0.029212,0.047609,0.000997,0.036907,0.086448,0.036975,0.062010,0.010845,0.022510,0.033514,0.104821,0.046757,0.071274,0.031502,0.036637,0.024994,0.019639,0.069599,0.018852,0.028783,0.091986,0.038078,0.073862,0.030001,0.033917,0.103003,0.049285,...,0.023919,0.030591,0.112242,0.133677,0.041828,0.033117,0.191406,0.119321,0.034152,0.141605,0.059687,0.026526,0.215738,0.064722,0.116646,0.041653,0.015912,0.068764,0.079212,0.036023,0.088103,0.017771,0.156516,0.020595,0.007492,0.012149,0.018611,0.064015,0.095532,0.072907,0.021596,0.076067,0.005073,0.015551,0.012973,0.121982,0.004994,0.027135,0.049045,0.055477
text_subjectivity,0.236463,0.007029,0.001274,1.000000,0.073185,0.063398,0.042495,0.007697,0.060223,0.005638,0.004968,0.050605,0.003650,0.086683,0.036969,0.118932,0.051700,0.038725,0.015315,0.038848,0.007517,0.008891,0.060806,0.085339,0.007562,0.033534,0.050886,0.018532,0.038234,0.119511,0.034231,0.002037,0.157462,0.043102,0.125347,0.065077,0.040683,0.102110,0.020624,0.055990,...,0.067773,0.040068,0.023078,0.011863,0.040448,0.184690,0.065208,0.017602,0.038156,0.038765,0.027668,0.025561,0.027297,0.003412,0.061948,0.062764,0.012059,0.050169,0.055710,0.007816,0.005478,0.031602,0.151203,0.082274,0.001615,0.056855,0.002066,0.006338,0.010163,0.019045,0.001155,0.047588,0.024855,0.038332,0.006348,0.039281,0.062913,0.010224,0.026297,0.063931
text_posneg,0.167387,0.052693,0.583386,0.073185,1.000000,0.184209,0.061268,0.043146,0.016397,0.006954,0.021911,0.015647,0.013653,0.054681,0.030580,0.014715,0.010864,0.063475,0.020209,0.064504,0.025656,0.006614,0.0

In [107]:
df_final.drop([column for column in upper.columns if any(upper[column] > 0.6)], axis=1, inplace=True)
# Now we separate in train/test sets
df_final.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,u_timestamp,target,text_polarity,text_subjectivity,text_posneg,VEC_abc,VEC_acceler,VEC_accept,VEC_access,VEC_accord,VEC_account,VEC_accus,VEC_acquir,VEC_acquisit,VEC_act,VEC_action,VEC_activ,VEC_ad,VEC_adam,VEC_add,VEC_address,VEC_adopt,VEC_advertis,VEC_affect,VEC_africa,VEC_african,VEC_agenc,VEC_agent,VEC_aggreg,VEC_ahead,VEC_aid,VEC_aim,VEC_air,VEC_airdrop,VEC_airport,VEC_algorithm,VEC_alleg,VEC_allow,VEC_alltim,VEC_altcoin,...,VEC_wallet,VEC_wanna,VEC_want,VEC_war,VEC_warn,VEC_watch,VEC_watchdog,VEC_way,VEC_wealth,VEC_web,VEC_websit,VEC_week,VEC_west,VEC_whale,VEC_white,VEC_wild,VEC_win,VEC_wirecard,VEC_wirex,VEC_withdraw,VEC_withdrawn,VEC_world,VEC_worldwid,VEC_worri,VEC_worst,VEC_worth,VEC_wrap,VEC_yam,VEC_year,VEC_yearold,VEC_yield,VEC_york,VEC_young,VEC_youtub,VEC_yuan,VEC_zcash,VEC_zebpay,VEC_zec,VEC_zero,VEC_zone
0,1.598634e+09,1,0.2023,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.598548e+09,0,0.9136,0.462500,5.656250,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.598461e+09,1,-0.5647,0.254167,0.762295,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
3,1.598375e+09,0,0.1531,0.518182,1.428571,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.598202e+09,0,0.8126,0.319886,3.653061,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [108]:
df_final.head()

,u_timestamp,target,text_polarity,text_subjectivity,text_posneg,VEC_abc,VEC_acceler,VEC_accept,VEC_access,VEC_accord,VEC_account,VEC_accus,VEC_acquir,VEC_acquisit,VEC_act,VEC_action,VEC_activ,VEC_ad,VEC_adam,VEC_add,VEC_address,VEC_adopt,VEC_advertis,VEC_affect,VEC_africa,VEC_african,VEC_agenc,VEC_agent,VEC_aggreg,VEC_ahead,VEC_aid,VEC_aim,VEC_air,VEC_airdrop,VEC_airport,VEC_algorithm,VEC_alleg,VEC_allow,VEC_alltim,VEC_altcoin,...,VEC_wallet,VEC_wanna,VEC_want,VEC_war,VEC_warn,VEC_watch,VEC_watchdog,VEC_way,VEC_wealth,VEC_web,VEC_websit,VEC_week,VEC_west,VEC_whale,VEC_white,VEC_wild,VEC_win,VEC_wirecard,VEC_wirex,VEC_withdraw,VEC_withdrawn,VEC_world,VEC_worldwid,VEC_worri,VEC_worst,VEC_worth,VEC_wrap,VEC_yam,VEC_year,VEC_yearold,VEC_yield,VEC_york,VEC_young,VEC_youtub,VEC_yuan,VEC_zcash,VEC_zebpay,VEC_zec,VEC_zero,VEC_zone
0,1.598634e+09,1,0.2023,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.598548e+09,0,0.9136,0.462500,5.656250,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.598461e+09,1,-0.5647,0.254167,0.762295,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
3,1.598375e+09,0,0.1531,0.518182,1.428571,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.598202e+09,0,0.8126,0.319886,3.653061,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [109]:
X = df_final.iloc[:,2:].values
y = df_final.iloc[:,1].values

In [110]:
div = int(round(len(X) * 0.2))
X_train = X[:div]
y_train = y[:div]
X_test = X[div:]
y_test = y[div:]
# Finally, we perform ML and see results

In [111]:
X_train[:10]

array([[ 0.2023    ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.9136    ,  0.4625    ,  5.65625   , ...,  0.        ,
         0.        ,  0.        ],
       [-0.5647    ,  0.25416667,  0.76229508, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.4588    ,  0.05      ,  2.33333333, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.2691    ,  0.48166667,  1.28358209, ...,  0.        ,
         0.        ,  0.        ],
       [-0.7351    ,  0.70666667,  0.47619048, ...,  0.        ,
         0.        ,  0.        ]])

In [112]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
forest = RandomForestClassifier(random_state = 0)

In [113]:
rf = RandomForestRegressor(n_estimators = 1000, random_state=0)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
df_res = pd.DataFrame({'y_test':y_test, 'y_pred':y_pred})

In [114]:
threshold = 0.5
preds = [1 if val > threshold else 0 for val in df_res['y_pred']]
print(metrics.confusion_matrix(preds, df_res['y_test']))
print('Accuracy Score:')
print(accuracy_score(preds, df_res['y_test']))
print('Precision Score:')
print(precision_score(preds, df_res['y_test']))

[[20 17]
 [72 80]]
Accuracy Score:
0.5291005291005291
Precision Score:
0.8247422680412371


In [115]:
n_estimators = [200, 500, 1000]
max_depth = [5, 15, 25, 30]
min_samples_split = [2, 5, 10,]
min_samples_leaf = [1, 2, 5, 10] 

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)

gridF = GridSearchCV(forest, hyperF, cv = 3, verbose = 1, 
                      n_jobs = -1)
bestF = gridF.fit(X_train, y_train)

Fitting 3 folds for each of 144 candidates, totalling 432 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   30.7s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 432 out of 432 | elapsed:  4.7min finished


In [116]:
bestF.best_params_

{'max_depth': 5,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 200}

In [117]:
forest_opt = RandomForestClassifier(max_depth=5,min_samples_leaf=1,min_samples_split=2,n_estimators=200,random_state = 0)

In [118]:
forest_opt.fit(X_train, y_train)
y_pred_opt = forest_opt.predict(X_test)
df_res_opt = pd.DataFrame({'y_test':y_test, 'y_pred_opt':y_pred_opt})

In [119]:
print(metrics.confusion_matrix(df_res_opt['y_pred_opt'], df_res_opt['y_test']))
print('Accuracy Score:')
print(accuracy_score(df_res_opt['y_pred_opt'], df_res_opt['y_test']))
print('Precision Score:')
print(precision_score(df_res_opt['y_pred_opt'], df_res_opt['y_test']))

[[ 0  0]
 [92 97]]
Accuracy Score:
0.5132275132275133
Precision Score:
1.0
